# OrangePi AIpro 中文文本分类 (MindSpore + MindSpore NLP)

## 组件版本要求
- Python 3.9
- CANN Toolkit/Kernels 8.0.0.beta1
- MindSpore Ascend 2.6.0
- MindSpore NLP 0.4.1

本 Notebook 结构：
1. 环境检测与依赖安装检查
2. 加载中文文本分类模型
3. 推理示例
4. 推理性能评估
5. 生成独立推理脚本与运行指令

## 1. 环境检测与依赖确认

In [3]:
# 说明:
# - 自动检查并安装所需 Python 包 (mindspore-ascend==2.6.0, mindnlp==0.4.1, tqdm)
# - 可通过环境变量覆盖安装命令:
#     MS_INSTALL_CMD  (默认: pip install mindspore-ascend==2.6.0 -i 清华源)
#     MINDNLP_INSTALL_CMD (默认: pip install mindnlp==0.4.1 -i 清华源)
# - 成功后打印版本与 Ascend 目标设备。

import sys, importlib, subprocess, os, platform, json, time, pathlib
REQ_PY = (3,9)
if sys.version_info[:2] != REQ_PY:
    raise SystemExit(f"需要 Python {REQ_PY}, 当前: {sys.version_info[:2]}")
print(f"Python {sys.version.split()[0]} OK")

DEFAULT_MS_CMD = "pip install mindspore-ascend==2.6.0 -i https://pypi.tuna.tsinghua.edu.cn/simple"
DEFAULT_MINDNLP_CMD = "pip install mindnlp==0.4.1 -i https://pypi.tuna.tsinghua.edu.cn/simple"
DEFAULT_TQDM_CMD = "pip install tqdm -i https://pypi.tuna.tsinghua.edu.cn/simple"

MS_WHL = os.environ.get('MS_WHL')
MINDNLP_WHL = os.environ.get('MINDNLP_WHL')
MS_INSTALL_CMD = os.environ.get('MS_INSTALL_CMD') or (f"pip install {MS_WHL}" if MS_WHL else DEFAULT_MS_CMD)
MINDNLP_INSTALL_CMD = os.environ.get('MINDNLP_INSTALL_CMD') or (f"pip install {MINDNLP_WHL}" if MINDNLP_WHL else DEFAULT_MINDNLP_CMD)

PKGS = [
    ("mindspore", "2.6.0", MS_INSTALL_CMD),
    ("mindnlp", "0.4.1", MINDNLP_INSTALL_CMD),
    ("tqdm", None, DEFAULT_TQDM_CMD),
]


def ensure_pkg(mod_name: str, want_ver: str, install_cmd: str):
    """确保模块存在且（若指定）版本匹配，不匹配则尝试安装。"""
    need_install = False
    try:
        m = importlib.import_module(mod_name)
        cur_ver = getattr(m, '__version__', '?')
        if want_ver and want_ver not in cur_ver:
            print(f"[INFO] {mod_name} 版本 {cur_ver} != 期望 {want_ver}, 准备重新安装")
            need_install = True
        else:
            print(f"[OK] {mod_name} 已满足 -> {cur_ver}")
    except Exception:
        print(f"[MISS] 未找到 {mod_name}, 准备安装")
        need_install = True
    if need_install:
        print(f"[INSTALL] {install_cmd}")
        r = subprocess.run(install_cmd, shell=True)
        if r.returncode != 0:
            raise SystemExit(f"安装 {mod_name} 失败，请检查网络或提供离线 whl")
        m = importlib.import_module(mod_name)
        print(f"[DONE] {mod_name} -> {getattr(m,'__version__','?')}")

for n, v, cmd in PKGS:
    ensure_pkg(n, v, cmd)

import mindspore as ms
print('MindSpore 版本:', ms.__version__)
try:
    import mindnlp
    print('MindNLP 版本:', mindnlp.__version__)
except Exception as e:
    print('MindNLP 导入失败:', e)

try:
    ms.set_context(device_target='Ascend')
    print('Device target:', ms.get_context('device_target'))
except Exception as e:
    print('设置 Ascend 失败，可能当前环境无 NPU 驱动:', e)

def find_opp_roots():
    candidates = [
        '/usr/local/Ascend/ascend-toolkit/latest/opp',
        '/usr/local/Ascend/opp',
        '/usr/local/ascend/ascend-toolkit/latest/opp',
    ]
    found = []
    for c in candidates:
        if os.path.isdir(c):
            found.append(c)
    return found

opp_roots = find_opp_roots()
if not opp_roots:
    print('\n[CRITICAL] 未找到 Ascend OPP 目录 (opp_kernel / op_info_cfg).')
    print('可能原因:')
    print('  1) CANN Toolkit 未完整安装 (缺少 ascend-toolkit 组件)')
    print('  2) 安装后未正确挂载 /usr/local/Ascend 到容器')
    print('  3) 权限或软链接缺失')
    print('\n建议操作 (任选适用):')
    print('  a) 确认已安装 CANN 8.0.0.beta1 ascend-toolkit 与 driver 包')
    print('  b) 重新执行 toolkit 安装: bash Ascend-cann-toolkit*.run --install')
    print('  c) 若在容器, 使用 --volume /usr/local/Ascend:/usr/local/Ascend 挂载宿主路径')
    print('  d) 检查权限: sudo chmod -R a+r /usr/local/Ascend')
    print('  e) 安装后需执行环境脚本: source /usr/local/Ascend/ascend-toolkit/set_env.sh')
    print('\n后续 MindSpore 需要读取 op_info_cfg/*.json 以注册算子，缺失会继续报错。')
else:
    print('\n发现 OPP 目录:')
    for r in opp_roots:
        op_cfg = os.path.join(r, 'op_info_cfg')
        has_cfg = os.path.isdir(op_cfg)
        print(f'  - {r} | op_info_cfg={"OK" if has_cfg else "缺失"}')
        if has_cfg:
            try:
                cnt = len([f for f in os.listdir(op_cfg) if f.endswith('.json')])
                print(f'    -> op_info_cfg 中 JSON 文件数: {cnt}')
            except Exception as e:
                print('    -> 统计失败:', e)

print('\n依赖检测/安装完成。')
# ===================================================================

Python 3.9.2 OK
[OK] mindspore 已满足 -> 2.6.0
[INFO] mindnlp 版本 ? != 期望 0.4.1, 准备重新安装
[INSTALL] pip install mindnlp==0.4.1 -i https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


[WARNING] ME(4693:255086074413088,MainProcess):2025-09-01-01:37:15.109.098 [mindspore/context.py:1402] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.


[DONE] mindnlp -> ?
[OK] tqdm 已满足 -> 4.67.1
MindSpore 版本: 2.6.0
MindNLP 导入失败: module 'mindnlp' has no attribute '__version__'
Device target: Ascend

发现 OPP 目录:
  - /usr/local/Ascend/ascend-toolkit/latest/opp | op_info_cfg=缺失

依赖检测/安装完成。


## 2. 加载中文文本分类模型

本节直接加载已经在公开中文数据集上完成微调的文本分类模型。

In [4]:
from mindnlp.transformers import AutoTokenizer, AutoModelForSequenceClassification

FINETUNED_CANDIDATES = [
    'uer/roberta-base-finetuned-chinanews-chinese',  # 中国新闻分类
    'clue/roberta_chinese_clue_tnews',               # TNEWS 主题分类
    'hfl/chinese-bert-wwm-ext',                     # 若上面都失败则使用未特定微调模型
]

NUM_LABELS = None  
USED_MODEL = None
model = tokenizer = None
for name in FINETUNED_CANDIDATES:
    try:
        print('尝试加载已微调模型:', name)
        tokenizer = AutoTokenizer.from_pretrained(name)
        model = AutoModelForSequenceClassification.from_pretrained(name)
        USED_MODEL = name
        print('[OK] 使用模型:', name)
        break
    except Exception as e:
        print('[Fail]', name, '->', e)

if model is None:
    raise SystemExit('未能加载任何已微调模型, 请检查网络或改为离线加载。')

model.set_train(False)
print('标签数:', getattr(model, 'num_labels', '未知'))

尝试加载已微调模型: uer/roberta-base-finetuned-chinanews-chinese


/usr/local/miniconda3/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


[OK] 使用模型: uer/roberta-base-finetuned-chinanews-chinese
标签数: 7


## 3. 推理示例

模型为已微调成品分类器：
1. 准备待分类文本列表。
2. tokenizer 编码 (padding / truncation)。
3. 前向 -> logits -> softmax -> argmax 得到标签。

下一个代码单元给出批量推理示例与辅助函数。

In [5]:
import numpy as np, mindspore as ms
from mindspore import Tensor, ops

MAX_LEN = 128

# 若模型自带 id2label 属性使用之, 否则构造占位标签映射
def build_label_maps(m):
    id2label = getattr(m.config, 'id2label', None) if hasattr(m, 'config') else None
    label2id = getattr(m.config, 'label2id', None) if hasattr(m, 'config') else None
    if id2label and isinstance(id2label, dict):
        # 规范化 key 为 int
        fixed = {}
        for k,v in id2label.items():
            try:
                fixed[int(k)] = v
            except: pass
        id2label = fixed
    else:
        n = getattr(m, 'num_labels', None) or 0
        id2label = {i: f'LABEL_{i}' for i in range(n)}
    if not label2id:
        label2id = {v:k for k,v in id2label.items()}
    return id2label, label2id

id2label, label2id = build_label_maps(model)
print('标签映射大小:', len(id2label))

def infer_texts(text_list):
    enc = tokenizer(text_list, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors=None)
    ids_np = np.array(enc['input_ids'], dtype=np.int32)
    attn_np = np.array(enc['attention_mask'], dtype=np.int32)
    tok_np  = np.zeros_like(ids_np, dtype=np.int32)
    ids = Tensor(ids_np); attn = Tensor(attn_np); tok = Tensor(tok_np)
    model.set_train(False)
    out = model(input_ids=ids, attention_mask=attn, token_type_ids=tok)
    probs = ops.softmax(out.logits, axis=-1).asnumpy()
    preds = probs.argmax(-1)
    results = []
    for i, p in enumerate(preds):
        label = id2label.get(int(p), str(p))
        conf = float(probs[i][p])
        results.append((text_list[i], label, conf))
    return results

# 示例
sample_texts = [
    '苹果公司发布新款智能手机，市场反应热烈',
    '国家出台教育改革政策，家长学生广泛关注',
    '昨晚的足球比赛十分激烈，球迷情绪高涨'
]
for t, lbl, conf in infer_texts(sample_texts):
    print(t[:24], '->', lbl, f'{conf:.4f}')

标签映射大小: 7
苹果公司发布新款智能手机，市场反应热烈 -> financial news 0.8379
国家出台教育改革政策，家长学生广泛关注 -> mainland China politics 0.8267
昨晚的足球比赛十分激烈，球迷情绪高涨 -> sports 0.9990


## 4. 推理性能评估

In [6]:
import time, numpy as np, mindspore as ms
from mindspore import Tensor, ops

perf_texts = [
    '苹果公司发布最新芯片，引发科技股关注',
    '国家出台新的教育政策，家长和学生高度关注',
    '国际足联公布最新排名，球队表现引热议',
    '新赛季篮球联赛即将开始，球迷购票火爆',
    '大型游戏公司发布新作预告，引玩家期待'
]
enc = tokenizer(perf_texts, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors=None)
ids = Tensor(np.array(enc['input_ids'], dtype=np.int32))
attn = Tensor(np.array(enc['attention_mask'], dtype=np.int32))
zero_tok = ops.zeros_like(ids)
model.set_train(False)

# 单句推理耗时
def time_single(n=30, warmup=5):
    single_ids = ids[0:1]; single_attn = attn[0:1]; single_tok = zero_tok[0:1]
    for _ in range(warmup): _ = model(input_ids=single_ids, attention_mask=single_attn, token_type_ids=single_tok)
    t0 = time.time()
    for _ in range(n): _ = model(input_ids=single_ids, attention_mask=single_attn, token_type_ids=single_tok)
    avg = (time.time() - t0) / n * 1000
    print(f'单句平均耗时: {avg:.2f} ms (基于 {n} 次)')

# 批量推理耗时
def time_batch(n=50, warmup=5):
    for _ in range(warmup): _ = model(input_ids=ids, attention_mask=attn, token_type_ids=zero_tok)
    t0 = time.time()
    for _ in range(n): _ = model(input_ids=ids, attention_mask=attn, token_type_ids=zero_tok)
    avg = (time.time() - t0) / n * 1000
    per_sample = avg / len(perf_texts)
    print(f'批量({len(perf_texts)})平均耗时: {avg:.2f} ms -> {per_sample:.2f} ms/样本')

out = model(input_ids=ids, attention_mask=attn, token_type_ids=zero_tok)
probs = ops.softmax(out.logits, -1).asnumpy(); preds = probs.argmax(-1)
for t, p in zip(perf_texts, preds):
    label = id2label.get(int(p), str(p))
    print(t[:18], '->', label)
try:
    time_single(); time_batch()
except Exception as e:
    print('性能计时失败:', e)

苹果公司发布最新芯片，引发科技股关注 -> financial news
国家出台新的教育政策，家长和学生高度 -> mainland China politics
国际足联公布最新排名，球队表现引热议 -> sports
新赛季篮球联赛即将开始，球迷购票火爆 -> sports
大型游戏公司发布新作预告，引玩家期待 -> financial news
单句平均耗时: 85.69 ms (基于 30 次)
批量(5)平均耗时: 103.42 ms -> 20.68 ms/样本


## 5. 生成独立推理脚本与运行指令

In [ ]:
# 运行本单元生成 inference_transformer_server.py
# 用法: python inference_transformer_server.py <host> <port> [model_name] [--max-len 128] [--label-file export/label_mapping.json]

# 直接设定默认模型（用户未指定时使用）；若运行时传入其他模型名则覆盖
MODEL_PLACEHOLDER = 'uer/roberta-base-finetuned-chinanews-chinese'
MAXLEN_PLACEHOLDER = MAX_LEN

script_template = r'''#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""Transformer 中文文本分类 Gradio 服务 (MindSpore + MindSpore NLP)

用法:
  python inference_transformer_server.py <host> <port> [model_name] [--max-len 128] [--label-file export/label_mapping.json]
示例:
  python inference_transformer_server.py 0.0.0.0 7860
  python inference_transformer_server.py 0.0.0.0 7860 uer/roberta-base-finetuned-chinanews-chinese
  python inference_transformer_server.py 0.0.0.0 7860 --max-len 256
  python inference_transformer_server.py 0.0.0.0 7860 --label-file export/label_mapping.json

特点:
  - 参考原 TextCNN server.py 的结构与输出样式
  - 自动从 label 文件或模型 config 回退获取标签
  - 仅文本显示 Top5 (去除图表组件，便于简化显示)
  - Ascend 设备优先 (可根据实际环境自动回退)
"""
import sys, os, json, time
import numpy as np
import mindspore as ms
from mindspore import Tensor
from mindnlp.transformers import AutoTokenizer, AutoModelForSequenceClassification

from argparse import ArgumentParser
parser = ArgumentParser()
parser.add_argument('host', help='监听地址，如 0.0.0.0')
parser.add_argument('port', type=int, help='端口，如 7860')
parser.add_argument('model_name', nargs='?', default='__MODEL_NAME__', help='HuggingFace 模型名称或本地目录')
parser.add_argument('--max-len', type=int, default=__MAX_LEN__, help='Tokenizer 截断/填充长度')
parser.add_argument('--label-file', default='export/label_mapping.json', help='标签映射 JSON (可选)')
parser.add_argument('--device', default='Ascend', choices=['Ascend','CPU','GPU'], help='设备优先级 (默认 Ascend)')
args = parser.parse_args()

HOST, PORT = args.host, args.port
MODEL_NAME = args.model_name
MAX_LEN = args.max_len
LABEL_FILE = args.label_file
DEVICE = args.device

# 设备设置
try:
    ms.set_context(device_target=DEVICE)
    print(f"[Context] device_target={ms.get_context('device_target')}")
except Exception as e:
    print('[Context][WARN] 设置指定设备失败, 尝试回退 CPU:', e)
    try:
        ms.set_context(device_target='CPU')
    except Exception as e2:
        print('[Context][FATAL] 无法设置任何设备:', e2)
        sys.exit(1)

# 标签加载
def load_id2label(model_obj):
    if LABEL_FILE and os.path.exists(LABEL_FILE):
        try:
            with open(LABEL_FILE,'r',encoding='utf-8') as f:
                data = json.load(f)
            raw = data.get('id2label') or {}
            id2label = {int(k):v for k,v in raw.items()}
            if id2label:
                print(f"[Label] 从文件加载: {LABEL_FILE} (共 {len(id2label)})")
                return id2label
            else:
                print('[Label][WARN] 文件格式不符或为空, 回退模型 config')
        except Exception as e:
            print('[Label][WARN] 文件读取失败, 回退模型 config:', e)
    cfg_map = getattr(getattr(model_obj,'config', None), 'id2label', None)
    if cfg_map and isinstance(cfg_map, dict) and cfg_map:
        try:
            id2label = {int(k):v for k,v in cfg_map.items()}
            print(f"[Label] 使用模型 config id2label (共 {len(id2label)})")
            return id2label
        except: pass
    n = getattr(model_obj,'num_labels', 0)
    id2label = {i: f'LABEL_{i}' for i in range(n)}
    print(f"[Label] 构造占位标签 (num_labels={n})")
    return id2label

print(f"[Init] 加载模型: {MODEL_NAME}")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
except Exception as e:
    print('[Init][FATAL] 模型加载失败:', e)
    sys.exit(1)
model.set_train(False)
id2label = load_id2label(model)

if LABEL_FILE and not os.path.exists(LABEL_FILE):
    try:
        os.makedirs(os.path.dirname(LABEL_FILE) or '.', exist_ok=True)
        with open(LABEL_FILE,'w',encoding='utf-8') as f:
            json.dump({'id2label': {str(i):lab for i,lab in id2label.items()}, 'label2id': {lab:i for i,lab in id2label.items()}}, f, ensure_ascii=False, indent=2)
        print(f"[Label] 已写出标签映射: {LABEL_FILE}")
    except Exception as e:
        print('[Label][WARN] 标签写出失败:', e)

import mindspore.ops as ops

def classify_text(text: str):
    if not text.strip():
        return '**提示**: 请输入文本'
    enc = tokenizer([text], truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors=None)
    ids = Tensor(np.array(enc['input_ids'], dtype=np.int32))
    attn = Tensor(np.array(enc['attention_mask'], dtype=np.int32))
    toks = Tensor(np.zeros_like(np.array(enc['input_ids'], dtype=np.int32)))
    out = model(input_ids=ids, attention_mask=attn, token_type_ids=toks)
    probs = ops.softmax(out.logits, axis=-1).asnumpy()[0]
    pred = int(probs.argmax())
    best_label = id2label.get(pred, str(pred))
    best_conf = float(probs[pred])
    ranking = [(id2label.get(i,str(i)), float(probs[i])) for i in range(len(probs))]
    ranking.sort(key=lambda x: x[1], reverse=True)
    lines = [f'**预测类别**: {best_label}', f'**置信度**: {best_conf:.4f}', '', '**Top5 概率:**']
    for i,(lab,pv) in enumerate(ranking[:5]):
        lines.append(f'{i+1}. {lab}: {pv:.4f}')
    return '\n'.join(lines)

print('[Gradio] 构建界面...')
try:
    import gradio as gr
except ImportError:
    print('[Gradio] 未安装 gradio, 正在尝试安装...')
    os.system('pip install gradio')
    import gradio as gr

examples = [
    '苹果公司发布新品手机，市场反应强烈',
    '国家队在世界杯预选赛中取得关键胜利',
    '人工智能技术在医疗影像诊断领域突破'
]

with gr.Blocks(title='Transformer 文本分类器') as demo:
    gr.Markdown('# Transformer 中文文本分类')
    gr.Markdown(f'模型: {MODEL_NAME} | 标签数: {len(id2label)} | MaxLen: {MAX_LEN}')
    with gr.Row():
        with gr.Column(scale=2):
            inp = gr.Textbox(label='输入文本', lines=5, placeholder='输入需要分类的中文文本...')
            btn = gr.Button('开始分类', variant='primary')
            gr.Examples(examples=examples, inputs=inp, label='示例')
        with gr.Column(scale=2):
            out_md = gr.Markdown()
    btn.click(classify_text, inputs=inp, outputs=out_md)
    gr.Markdown('---')
    gr.Markdown('推理由 MindSpore + MindNLP 驱动，界面基于 Gradio。')

print(f'[Gradio] 启动: http://{HOST}:{PORT}')
try:
    demo.launch(server_name=HOST, server_port=PORT, share=False, show_error=True)
except Exception as e:
    print('[Gradio][FATAL] 启动失败:', e)
    sys.exit(1)
'''

script_content = script_template.replace('__MODEL_NAME__', MODEL_PLACEHOLDER).replace('__MAX_LEN__', str(MAXLEN_PLACEHOLDER))

out_file = 'inference_transformer_server.py'
with open(out_file,'w',encoding='utf-8') as f:
    f.write(script_content)

print(f'已生成 {out_file} (默认模型: {MODEL_PLACEHOLDER})')
print('运行示例:')
print('  python inference_transformer_server.py 0.0.0.0 7860')
print('  python inference_transformer_server.py 0.0.0.0 7860 clue/roberta_chinese_clue_tnews')
print('  python inference_transformer_server.py 0.0.0.0 7860 --max-len 256')
print('  python inference_transformer_server.py 0.0.0.0 7860 --label-file export/label_mapping.json')

已生成 inference_transformer_server.py (默认模型: uer/roberta-base-finetuned-chinanews-chinese)
运行示例:
  python inference_transformer_server.py 0.0.0.0 7860
  python inference_transformer_server.py 0.0.0.0 7860 clue/roberta_chinese_clue_tnews
  python inference_transformer_server.py 0.0.0.0 7860 --max-len 256
  python inference_transformer_server.py 0.0.0.0 7860 --label-file export/label_mapping.json
